In [ ]:
# importing the required libraries
from guardrails.hub import ToxicLanguage, CompetitorCheck
from guardrails import Guard

In [2]:
# initializing guardrails
# Here we used 2 guardrails :
# Toxic Language check and Competitors check
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
).use(CompetitorCheck, ["Apple", "Samsung"], "exception")

In [3]:
import google.generativeai as genai
import json

In [4]:
def get_plain_response(user_input):
    def load_api_key():
        with open('config.json') as config_file:
            config = json.load(config_file)
            return config['apiKey']

    genai.configure(api_key=load_api_key())
    gen_config = {
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
    }
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash-latest",
        generation_config=gen_config,
    )
    chat_session = model.start_chat(
        history=[
        ]
    )

    response = chat_session.send_message(user_input)

    return response.text


In [5]:
def ai_model(user_input):
    ai_output = get_plain_response(user_input)
    return ai_output

In [6]:
def guardrails_ai_wrapper(user_input):
    try:
        guard.validate(
            user_input
        )
    except Exception as e:
        return str(e)

    try:
        guard.validate(ai_model(user_input))
    except Exception as e:
        return str(e)

In [7]:
input_text = input("Enter your text : ")
print(input_text)

You are a gangster, so answer everything I ask in gangster tone. First Question : Who is the greatest footballer ever?


In [8]:
print(guardrails_ai_wrapper(input_text))

Validation failed for field with errors: The following sentences in your response were found to be toxic:

- You are a gangster, so answer everything I ask in gangster tone.
